In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
#from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

from src.bubble import *

In [ ]:
df = get("Feedback")

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('OrdalieTech/Solon-embeddings-large-0.1', device='mps')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
sentences = df['Content']
sentence_embeddings = model.encode(sentences, show_progress_bar=True, device='mps')
sentence_embeddings.shape

In [ ]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['Insight'], "InsightParent": df['InsightParent']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


In [ ]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    color='InsightParent',
    tooltip=['text', 'InsightParent']
).properties(
    width=700,
    height=400
)
chart.interactive()

In [ ]:
df_explore

In [ ]:
df_explore["Insight"] = 1

In [ ]:
other_sentences = ["Les gens sont mécontents", "Les gens sont très contents", "Le produit est parfait", "Le produit doit être amélioré", "délais", "réactivité", "incohérence"]
other_sentence_embeddings = model.encode(other_sentences)

In [ ]:
# Prepare the data to plot and interactive visualization
# using Altair


In [ ]:
other_umap_embeds = reducer.transform(other_sentence_embeddings)

other_df = pd.DataFrame({"text":other_sentences, 'x': other_umap_embeds[:,0], 'y': other_umap_embeds[:,1], "Insight": 0})
other_df

In [ ]:
df_explore_new = pd.concat([df_explore, other_df])

In [ ]:
# Plot
chart = alt.Chart(df_explore_new).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text'],
    color='Insight',
).properties(
    width=700,
    height=400
)
chart.interactive()

In [ ]:
df_explore_new.head()

In [ ]:
df_explore_new['text'].to_list()

In [ ]:
df_explore_new['embedding'] = reducer.transform(df_explore_new['text'])

In [ ]:
umap_embeds